**Пример развертывания с помощью docker базы данных postgres взаимодейсвующей с программой на python**

# Разные контейнеры

Програма в одном контейнере, база в другом. В теории так и должно быть, так что это самый правильный вариант.

Запуск контейнеров

In [1]:
%%bash

# сеть
docker network create test_project_net
# postgres
docker run --rm -d\
    --name database\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    --net=test_project_net\
    custom_postgres
# python
docker run --rm -itd\
    --name python_psql\
    --net=test_project_net\
    python_psql

34a398d3020100bfba5156db3a78aef6f35d6c56051710a8178075f65b0e73cd
fb428a7b940e7ebf4c6eadd4d8e6c55e8c0f0125ca551bcbd350df65e0c212fb
8f717a2ad9f6998a2eb7db3ea0ac03cc96522b8a361db671c42bbdc8e9991a04


Надо создать таблицу с которой наша программа на python будет работать.

In [2]:
%%bash
# надо дождаться пока база данных
# развернется (но вообще это не обязательный шаг)
sleep 1
docker exec database \
    psql --username docker_app --dbname docker_app_db -f create_table.sql

CREATE TABLE


In [3]:
%%bash
docker exec python_psql python3 script.py

Проверим, чтобы записи сидели в базе.

In [4]:
%%bash 
docker exec database psql \
    --username docker_app \
    --dbname docker_app_db \
    -f create_table.sql\
    -c 'SELECT * FROM main_table;'

psql:create_table.sql:4: ERROR:  relation "main_table" already exists


 id |         text         
----+----------------------
 0  | vymszdacizasoskeynrz
 1  | zeedscxslqsviocgulpv
 2  | pgaddokdpeaetjcjbgxn
 3  | saofcamywgifngsrvghz
 4  | anjfbasvwncfsybackza
 5  | iuqjpjfxqzkcyjhxwixk
 6  | uxsfbomwzpjmoldpepvx
 7  | wkqqpzplzjbbuvyxnrxe
 8  | jrbmbnnpxmexjtiyuici
 9  | kllkgouehckewkmxsprv
 10 | gszezoecjruwnfrgtczc
 11 | mbudorfpbhmcklxhlyra
 12 | xbkhupkusdsswbqcxjkh
 13 | lsjyaooxxbouhoqfupzz
 14 | vymzfuuaaiwsqlfnpiuw
 15 | rqhcxlgefvhxgwsuyobn
 16 | canvcvnttbctysiczkmf
 17 | pbrlbrurbudorrcggsga
 18 | qvpslycpamqdwrxwpkpu
 19 | psiallsnotlotxeizopq
(20 rows)



Удаление всего того, что насоздавали.

In [5]:
%%bash
docker stop database
docker stop python_psql
docker network rm test_project_net

database
python_psql
test_project_net


# Программа на хосте

В целях отладки программы бывает полезно поднять контейнер с базой а программу отлаживать на localhost. В данном разделе представлено именно такой случай.

Старт контейнера с базой и создание табилцы.

In [7]:
%%bash
# особое внимане порту - тут от принципиально важен
docker run --rm -d\
    --name database\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    -p 5431:5432\
    custom_postgres

# надо дождаться пока база данных
# развернется (но вообще это не обязательный шаг)
sleep 1
docker exec database \
    psql --username docker_app --dbname docker_app_db -f create_table.sql

7b2c720f625cfe5edcbb933c177b3100e37804f546c67d154bdeaa8199909546
CREATE TABLE


В python надо открыть connection с базой.

In [8]:
import psycopg2
import random
import string

conn = psycopg2.connect(
    port = "5431",# указываем порт
    dbname = "docker_app_db",
    user = "docker_app",
    password = "docker_app",
    host= "localhost"
)

Вставляем в новосозданную табличку 20 новый записей.

In [9]:
cur = conn.cursor()
for i in range(20):
    text = ''.join(random.choices(string.ascii_lowercase, k=20))
    query = f"INSERT INTO main_table (id, text) VALUES ('{i}', '{text}');"
    cur.execute(query)
cur.close()

И посмотрим, что это за записи.

In [10]:
cur = conn.cursor()
cur.execute("SELECT * FROM main_table;")
for i in cur:
    print(i)
cur.close()

('0', 'jvzxfsnhqdqwzlcydplk')
('1', 'dkikwyqonxbpuntctwbh')
('2', 'hezacpzkyunfveeryric')
('3', 'tiyiamkwkpldwwcssybh')
('4', 'mlubzprlazklkwhfkjes')
('5', 'cljklgwdqrdixouxmwvx')
('6', 'qjwaaacfxtfgddfuuwcm')
('7', 'edchlgwxfleemxiygppw')
('8', 'jlhoegxnoufwwkmuqmbt')
('9', 'inghkupkzzbqwwsorbty')
('10', 'aarzufaztxozfknqnkry')
('11', 'rseoefecuwmukfsctvvc')
('12', 'jcndoogfkkfxrmhcynpn')
('13', 'noyhhyphzwyxbmggethp')
('14', 'klfzmuwbdikladtclngb')
('15', 'yqqrnsmrpzmmwxmhyulr')
('16', 'zfopjuwfzktsasfzwrbd')
('17', 'zdawvtlfpffdukteteys')
('18', 'zkculqcoqakiojcenslf')
('19', 'jzdhhvqmoghjjattawqj')


Отправляем изменения на базу и закрываем соединение.

In [11]:
conn.commit()
conn.close()

Для того, чтобы убедится, что все сработало, спрошу эту табличку напрямую из базы.

In [12]:
%%bash
docker exec database \
    psql --username docker_app --dbname docker_app_db -c 'SELECT * FROM main_table;'

 id |         text         
----+----------------------
 0  | jvzxfsnhqdqwzlcydplk
 1  | dkikwyqonxbpuntctwbh
 2  | hezacpzkyunfveeryric
 3  | tiyiamkwkpldwwcssybh
 4  | mlubzprlazklkwhfkjes
 5  | cljklgwdqrdixouxmwvx
 6  | qjwaaacfxtfgddfuuwcm
 7  | edchlgwxfleemxiygppw
 8  | jlhoegxnoufwwkmuqmbt
 9  | inghkupkzzbqwwsorbty
 10 | aarzufaztxozfknqnkry
 11 | rseoefecuwmukfsctvvc
 12 | jcndoogfkkfxrmhcynpn
 13 | noyhhyphzwyxbmggethp
 14 | klfzmuwbdikladtclngb
 15 | yqqrnsmrpzmmwxmhyulr
 16 | zfopjuwfzktsasfzwrbd
 17 | zdawvtlfpffdukteteys
 18 | zkculqcoqakiojcenslf
 19 | jzdhhvqmoghjjattawqj
(20 rows)



Останавливаю созданный для этого примера контейнер.

In [13]:
%%bash
docker stop database

database
